In [2]:
import subprocess
import os
image_dir = "/Users/jasper/Downloads/Video_rotations"

output_file = os.path.join(image_dir, "rotation.mp4")
command = [
    "ffmpeg",
    "-framerate", str(60),
    "-i", os.path.join(image_dir, "rotation_%d.png"),
    "-c:v", "libx264",
    '-pix_fmt', 'yuv420p',
    output_file
]

subprocess.run(command)

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with clang version 4.0.1 (tags/RELEASE_401/final)
  configuration: --prefix=/Users/jasper/opt/anaconda3/envs/MAIN_environment --cc=x86_64-apple-darwin13.4.0-clang --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55.  5.100 / 55.  5.100
Input #0, im

CompletedProcess(args=['ffmpeg', '-framerate', '60', '-i', '/Users/jasper/Downloads/Video_rotations/rotation_%d.png', '-c:v', 'libx264', '-pix_fmt', 'yuv420p', '/Users/jasper/Downloads/Video_rotations/rotation.mp4'], returncode=0)